In [1]:
from GPyOpt.methods import BayesianOptimization
import numpy as np
from numpy.random import randn
import time
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
np.random.seed(24)

In [3]:
def one_dim(x):
    #1-D
    #optimum at x=0.96609, f(x)=-1.48907
    return -(1.4-3*x)*np.sin(18*x)+randn()*sigma
bounds_one_dim = [{'name':'x', 'type':'continuous', 'domain':(0, 1.2)}]

In [4]:
def booth(X):
    #2-D
    #optimum at X=[1,3] f(X)=0
    X = np.atleast_2d(X)
    x = X[:, 0]
    y = X[:, 1]
    return((x+2*y-7)**2+(2*x+y-5)**2)+randn()*sigma
bounds_booth = [{'name': 'x', 'type': 'continuous', 'domain': (-10,10)}]

In [5]:
def sphere(X):
    #n-D
    #optimum at X = [0,...,0], f(X)=0
    X = np.atleast_2d(X)
    return(np.sum(np.power(X,2), 1))+randn()*sigma
bounds_sphere = [{'name': 'x', 'type': 'continuous', 'domain': (-1,1)}]

In [6]:
def rastrigin(X):
    #n-D
    #optimum at X=[0,...,0], f(X)=0
    X = np.atleast_2d(X)
    return(10*len(X[0])+np.sum(np.power(X,2)-10*np.cos(2*np.pi*X),1))+randn()*sigma
bounds_rastrigin = [{'name': 'x', 'type': 'continuous', 'domain': (-5.12,5.12)}]

Hyperparameter testing done in gpyopt_test.py

In [ ]:
# demonstrate with 1D - acquisition, noise effect, convergence
sigma=0
xdata = np.linspace(0, 1.2, 100)
ydata = [one_dim(x) for x in xdata]
plt.plot(xdata, ydata)
plt.show()
sigma = 0.1
myOpt = BayesianOptimization(one_dim, domain=bounds_one_dim, exact_feval=True)
myOpt.run_optimization(max_iter=50, max_time=60, eps=1e-6)
myOpt.plot_acquisition()
myOpt.plot_convergence()
print(myOpt.x_opt, myOpt.fx_opt)

In [ ]:
# show convergence for high dimension optimisation
sigma = 0
n=12
myOpt = BayesianOptimization(rastrigin, domain=bounds_rastrigin*n, exact_feval=True)
myOpt.run_optimization(max_iter=500, eps=1e-6)
myOpt.plot_convergence()
print(myOpt.x_opt, myOpt.fx_opt)

In [13]:
# get optimisation timing as number of iterations increases
results = pd.DataFrame(None)

n = 10
sigma = 0
max_time = 600
actual_x = [[0]*n]
actual_f = 0
acq_type = "LCB"

for seed in [1,2,3,4,5]:
    np.random.seed(seed)
    for function in [sphere, rastrigin]:
        if function == sphere:
            domain = bounds_sphere*n
            func_name = "sphere"
        else:
            domain = bounds_rastrigin*n
            func_name = "rastrigin"
        
        for max_iter in [50, 70, 90, 110, 130, 150]:
            myOpt = BayesianOptimization(function,
                                        domain=domain,
                                        acquisition_type=acq_type,
                                        exact_feval=True)
            myOpt.run_optimization(max_iter = max_iter, max_time = max_time, eps=1e-6)
            
            result = {"n": n,"function":func_name, "sigma": sigma, "max_it": max_iter, "max_time": max_time, "acquisition func": acq_type, 
                    "eucl_dist to true x_opt": float(np.linalg.norm(myOpt.x_opt-actual_x)), 
                    "diff to true f(x_opt)": float(np.abs(function(myOpt.x_opt)-actual_f)), "actual_it": myOpt.num_acquisitions,
                    "actual_time": myOpt.cum_time}
    
            results = results.append(result, ignore_index=True)
            print(myOpt.x_opt, result)


[-0.00329089  0.00079651  0.00576259 -0.00122352  0.00094411  0.00286922
 -0.00053055 -0.002933    0.00235334 -0.00241114] {'n': 10, 'function': 'sphere', 'sigma': 0, 'max_it': 50, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 0.008690701404103529, 'diff to true f(x_opt)': 7.552829089528704e-05, 'actual_it': 50, 'actual_time': 10.97363805770874}
[ 0.00405081 -0.00574328  0.00768452 -0.00276936  0.00519095  0.00198687
 -0.00781332 -0.00429094 -0.00131631  0.00026437] {'n': 10, 'function': 'sphere', 'sigma': 0, 'max_it': 70, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 0.015108669956903848, 'diff to true f(x_opt)': 0.0002282719078666489, 'actual_it': 70, 'actual_time': 16.604354858398438}
[-8.95152106e-04  5.43038431e-03 -3.80694537e-03  7.04380039e-03
 -9.07986306e-03 -6.61334674e-03  9.69889326e-05 -3.88002457e-05
  9.89950983e-04 -2.90126203e-03] {'n': 10, 'function': 'sphere', 'sigma': 0, 'max_it': 90, 'max_time': 600, 'acquisition f

[-0.16157447  0.06942965 -1.14486163 -0.87170773  2.0988701   0.84556332
 -1.07665971 -0.96586545 -1.0458664   3.04358312] {'n': 10, 'function': 'rastrigin', 'sigma': 0, 'max_it': 130, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 4.435207913857802, 'diff to true f(x_opt)': 40.643995302008946, 'actual_it': 130, 'actual_time': 37.042019844055176}
[ 2.9069937  -2.13541273  3.02911268 -1.02106902 -1.96026571 -0.04037588
  2.05215183  2.06533612  0.88740404  1.98623602] {'n': 10, 'function': 'rastrigin', 'sigma': 0, 'max_it': 150, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 6.346852538527591, 'diff to true f(x_opt)': 50.032928581468326, 'actual_it': 150, 'actual_time': 49.78085207939148}
[ 0.00499552 -0.00100994  0.00312182  0.00024215  0.00492225 -0.0023011
  0.00225405 -0.00176083 -0.00680267 -0.00283959] {'n': 10, 'function': 'sphere', 'sigma': 0, 'max_it': 50, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 0.01

[ 2.0956074   1.02278394 -0.89244498 -0.92008344 -2.92517607 -1.90801625
  2.93852652 -2.04522612  0.95322978 -0.03627246] {'n': 10, 'function': 'rastrigin', 'sigma': 0, 'max_it': 90, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 5.745058806383934, 'diff to true f(x_opt)': 42.82506943562774, 'actual_it': 90, 'actual_time': 21.79994797706604}
[-1.98872656  1.9533658   1.94609238  1.99233449 -0.97895326  0.00298624
  0.97605765  1.99243187 -2.94490726 -0.04717353] {'n': 10, 'function': 'rastrigin', 'sigma': 0, 'max_it': 110, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 5.484786168662585, 'diff to true f(x_opt)': 32.35647177211163, 'actual_it': 110, 'actual_time': 33.51465702056885}
[-0.0088978   0.978987   -2.04986889  3.0228368  -0.0055889  -0.01378869
  0.0062662  -0.04727745 -1.98206715  1.00984488] {'n': 10, 'function': 'rastrigin', 'sigma': 0, 'max_it': 130, 'max_time': 600, 'acquisition func': 'LCB', 'eucl_dist to true x_opt': 4.38

In [14]:
results.to_excel("gpyopt_temp_results.xlsx", index=False)

Sparse GP tested in gpyopt_test.py